1. Crea un modelo de clasificación del dataset MNIST con PyTorch con redes neuronales sin capas convolucionales, intentando mejorar todo lo posible su exactitud.

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Descarga el dataset MNIST y lo transforma a tensores
test_data = datasets.MNIST(
    root="data",         # Carpeta donde se almacenarán los datos
    train=False,         # Indica que es el conjunto de prueba
    download=True,       # Descarga el dataset si no está disponible
    transform=ToTensor() # Transforma las imágenes a tensores
)

training_data = datasets.MNIST(
    root="data",         # Carpeta donde se almacenarán los datos
    train=True,          # Indica que es el conjunto de entrenamiento
    download=True,       # Descarga el dataset si no está disponible
    transform=ToTensor() # Transforma las imágenes a tensores
)

# Define el DataLoader para los datos de entrenamiento y prueba
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False)

# Imprime la forma de los datos y los targets del conjunto de entrenamiento
print(training_data.data.shape)    # Imprime las dimensiones de las imágenes
print(training_data.targets.shape) # Imprime las dimensiones de las etiquetas

# Define la arquitectura de la red neuronal mejorada
class ImprovedNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # Capa para aplanar las imágenes de 2D a 1D
        self.flatten = nn.Flatten()
        # Define una secuencia de capas lineales, normalización, activación y dropout
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 1024),   # Capa lineal de 784 a 1024 neuronas
            nn.BatchNorm1d(1024),     # Normalización por lotes
            nn.ReLU(),                # Función de activación ReLU
            nn.Dropout(0.5),          # Dropout con probabilidad de 0.5
            nn.Linear(1024, 512),     # Capa lineal de 1024 a 512 neuronas
            nn.BatchNorm1d(512),      # Normalización por lotes
            nn.ReLU(),                # Función de activación ReLU
            nn.Dropout(0.5),          # Dropout con probabilidad de 0.5
            nn.Linear(512, 256),      # Capa lineal de 512 a 256 neuronas
            nn.BatchNorm1d(256),      # Normalización por lotes
            nn.ReLU(),                # Función de activación ReLU
            nn.Linear(256, 10)        # Capa lineal de 256 a 10 neuronas (salida)
        )

    def forward(self, x):
        x = self.flatten(x)               # Aplana la entrada
        logits = self.linear_relu_stack(x) # Pasa la entrada por las capas
        return logits                     # Retorna los logits (salidas)

# Inicializa el modelo
model = ImprovedNeuralNetwork()

# Selecciona el dispositivo de cómputo (GPU si está disponible, sino CPU)
device = (
    "cuda" if torch.cuda.is_available() 
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Mueve el modelo al dispositivo seleccionado
model = model.to(device)
print(model)

# Define la función de pérdida y el optimizador
loss_fn = nn.CrossEntropyLoss()          # Función de pérdida de entropía cruzada
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) # Optimizador Adam

# Define un scheduler para disminuir la tasa de aprendizaje
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Función de entrenamiento
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)     # Tamaño del dataset
    model.train()                      # Pone el modelo en modo entrenamiento
    for batch_num, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device) # Mueve datos y etiquetas al dispositivo

        pred = model(X)                # Calcula las predicciones
        loss = loss_fn(pred, y)        # Calcula la pérdida

        optimizer.zero_grad()          # Limpia los gradientes
        loss.backward()                # Retropropaga la pérdida
        optimizer.step()               # Actualiza los parámetros del modelo

        # Imprime la pérdida cada 100 batches
        if batch_num % 100 == 0:
            loss, current = loss.item(), (batch_num + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# Función de prueba
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)     # Tamaño del dataset
    num_batches = len(dataloader)      # Número de batches
    model.eval()                       # Pone el modelo en modo evaluación
    test_loss, correct = 0, 0          # Inicializa pérdida y aciertos
    with torch.no_grad():              # Desactiva el cálculo de gradientes
        for X, y in dataloader:
            X, y = X.to(device), y.to(device) # Mueve datos y etiquetas al dispositivo
            pred = model(X)           # Calcula las predicciones
            test_loss += loss_fn(pred, y).item() # Acumula la pérdida
            correct += (pred.argmax(1) == y).type(torch.float).sum().item() # Cuenta aciertos
    test_loss /= num_batches          # Promedia la pérdida
    correct /= size                   # Calcula la precisión
    accuracy = correct * 100          # Convierte a porcentaje
    print(f"Test Error: \n Accuracy: {accuracy:>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return accuracy, test_loss        # Retorna precisión y pérdida promedio

# Implementa Early Stopping
best_accuracy = 0                      # Inicializa la mejor precisión
epochs = 20                            # Número de épocas
patience = 3                           # Número de épocas para early stopping
trigger_times = 0                      # Contador de activaciones de early stopping

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer) # Entrena el modelo
    accuracy, avg_loss = test(test_dataloader, model, loss_fn) # Prueba el modelo
    
    # Actualiza el scheduler
    scheduler.step()
    
    # Early Stopping
    if accuracy > best_accuracy:
        best_accuracy = accuracy        # Actualiza la mejor precisión
        trigger_times = 0               # Resetea el contador
    else:
        trigger_times += 1              # Incrementa el contador
        print(f'Early stopping trigger times: {trigger_times}')
        
        if trigger_times >= patience:   # Si se alcanza el límite de paciencia
            print('Early stopping!')
            break                       # Detiene el entrenamiento

print(f"Best Accuracy: {best_accuracy:>0.1f}%")
print("Done!")


In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Definimos las transformaciones para preprocesado de las imágenes
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # Normalización con media y desviación estándar de MNIST
])

# Cargamos el dataset MNIST con las transformaciones definidas
train_data = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST('./data', train=False, download=True, transform=transform)

# Creamos los DataLoaders para los conjuntos de entrenamiento y prueba
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

from torch.nn import functional as F

# Definimos la arquitectura de la red neuronal convolucional mejorada
class ImprovedCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)   # Primera capa convolucional
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # Segunda capa convolucional
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1) # Tercera capa convolucional
        self.pool = nn.MaxPool2d(2, 2)  # Capa de Max Pooling
        self.dropout = nn.Dropout(0.25) # Capa de Dropout para regularización
        self.fc1 = nn.Linear(3 * 3 * 128, 256) # Primera capa totalmente conectada
        self.fc2 = nn.Linear(256, 128)         # Segunda capa totalmente conectada
        self.fc3 = nn.Linear(128, 10)          # Capa de salida

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) # Conv1 -> ReLU -> Pooling
        x = self.pool(F.relu(self.conv2(x))) # Conv2 -> ReLU -> Pooling
        x = self.pool(F.relu(self.conv3(x))) # Conv3 -> ReLU -> Pooling
        x = self.dropout(x)                  # Dropout
        x = x.view(-1, 3 * 3 * 128)          # Aplanamiento tras las capas convolucionales
        x = F.relu(self.fc1(x))              # FC1 -> ReLU
        x = self.dropout(x)                  # Dropout
        x = F.relu(self.fc2(x))              # FC2 -> ReLU
        x = self.fc3(x)                      # FC3 (salida)
        return F.log_softmax(x, dim=1)       # Log Softmax para usar con entropía cruzada

# Instanciamos la red neuronal mejorada
model = ImprovedCNN()
criterion = nn.CrossEntropyLoss() # Definimos la función de pérdida
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) # Definimos el optimizador Adam

# Seleccionamos el dispositivo (GPU si está disponible, sino CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device) # Movemos el modelo al dispositivo

# Entrenamiento del modelo
model.train()
num_epochs = 10

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device) # Movemos datos y etiquetas al dispositivo

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels) # Calculamos la pérdida

        # Backward pass y optimización
        optimizer.zero_grad() # Limpiamos los gradientes
        loss.backward()       # Retropropagamos la pérdida
        optimizer.step()      # Actualizamos los parámetros del modelo

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Evaluación del modelo
model.eval() # Ponemos el modelo en modo evaluación

correct = 0
total = 0
with torch.no_grad(): # Desactivamos el cálculo de gradientes
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1) # Predicción de la clase con mayor probabilidad
        total += labels.size(0)
        correct += (predicted == labels).sum().item() # Contamos los aciertos

accuracy = 100 * correct / total # Calculamos la precisión
print(f'Accuracy of the network on the 10000 test images: {accuracy:.2f}%')


Epoch [1/10], Step [100/938], Loss: 0.2587
Epoch [1/10], Step [200/938], Loss: 0.1111
Epoch [1/10], Step [300/938], Loss: 0.1187
Epoch [1/10], Step [400/938], Loss: 0.0913
Epoch [1/10], Step [500/938], Loss: 0.1477
Epoch [1/10], Step [600/938], Loss: 0.3752
Epoch [1/10], Step [700/938], Loss: 0.0897
Epoch [1/10], Step [800/938], Loss: 0.0551
Epoch [1/10], Step [900/938], Loss: 0.0789
Epoch [2/10], Step [100/938], Loss: 0.0990
Epoch [2/10], Step [200/938], Loss: 0.0510
Epoch [2/10], Step [300/938], Loss: 0.0222
Epoch [2/10], Step [400/938], Loss: 0.0880
Epoch [2/10], Step [500/938], Loss: 0.0276
Epoch [2/10], Step [600/938], Loss: 0.0390
Epoch [2/10], Step [700/938], Loss: 0.0357
Epoch [2/10], Step [800/938], Loss: 0.0080
Epoch [2/10], Step [900/938], Loss: 0.0468
Epoch [3/10], Step [100/938], Loss: 0.0056
Epoch [3/10], Step [200/938], Loss: 0.0122
Epoch [3/10], Step [300/938], Loss: 0.0444
Epoch [3/10], Step [400/938], Loss: 0.0679
Epoch [3/10], Step [500/938], Loss: 0.0782
Epoch [3/10